###     Maximizer's Choice

Innate greedy algorithm (polarization):

For each $i$ from $1$ to $k$, we choose $v_i$ such that:

$$v_i = argmax_{v \in V - \hat{s}} \max \left(\bar{s}_1^T \bar{s}_1, \bar{s}_0^T \bar{s}_0 \right) $$

In [1]:
# determines if value of opinion at v should be set to 0 or 1 to maximize equilibrium polarization 
def maximize_equi_polarization(op, n, v1):
    
    innat_equi_por = obj_innate_polarization(op, n)
    temp = op[v1, 0]
    por_arr = np.zeros(2)
#     print(por_arr)
    
#     max_opi_option = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    max_opi_option = [0.0, 1.0]
    
    # objective if set opinion to 0, 0.1, ..., 1.0
    i = 0
    for new_op in max_opi_option:
#         print('change op to '+ str(i/10))
        
        op[v1, 0] = new_op
        por_arr[i] = obj_polarization(A, L, op, n)
#         print(por_arr[i])
        i = i + 1
        op[v1, 0] = temp
#     print(por_arr)
    
    
    maxmize_op = np.argmax(por_arr)
    max_por = np.max(por_arr)
 
#     print('new_op', 'innat_equi_por', 'max_por')
#     print(maxmize_op, innat_equi_por, max_por)

    return (maxmize_op, innat_equi_por, max_por)

In [2]:
def choose_max_vertex(op, n, min_touched):
    # iterate over all the vertices that have not yet been changed
    #vertices = np.where((op != 0.0) & (op != 1.0))
    # current best vertex, its opinion {0, 1}, "innate" objective, and objective
    
    innat_equi_por = obj_polarization(A, L, op, n)
#     print("innate equi por to compare")
#     print(innat_equi_por)
    
    champion = (None, None, 0, innat_equi_por)
    
    start = time.perf_counter()
    for v1 in c1:
        print('Max start with agent '+ str(v1) )
        if v1 not in min_touched:
            (maxmize_op, innat_equi_por, max_por) = maximize_equi_polarization(op, n, v1)

            if max_por >= champion[3]:
                champion = (v1, maxmize_op, innat_equi_por, max_por)
    end = time.perf_counter()
    print(f'Finished in {round(end-start, 2)} second(s)') 
    return (champion)


### Minimizer's Choice

In [13]:
# determines if a given vertex should be set to zero or one, all else equal
def minimize_opinion(s, n, v2):
    
    # minimizer's options
    #min_opi_option = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    minimize_op = derivate_s(s, n, v2)# find min_s_star for each max_action
    
    op = copy.copy(s)
    op[v2, 0] = minimize_op
    
    min_pol = obj_polarization(A, L, op, n)

    return (minimize_op, min_pol)

In [15]:
def choose_min_vertex(s, n, max_touched):
    # current best vertex, its opinion {0, 1}, "innate" objective, and  current best objective
    
    innat_equi_por = obj_polarization(A, L, s, n)
#     print("innate equi por to compare")
#     print(innat_equi_por)

    champion = (None, None, 0, innat_equi_por)
    
    for v2 in c1:
        if v2 not in max_touched:
            print('Min start with agent '+ str(v2) )
#             print('Current opinion array')
#             print(op)
            
            (minimize_op, min_pol) = minimize_opinion(s, n, v2)  
#             print('minimize opinion')
#             print(minimize_op, min_pol)
            if min_pol <= champion[3]:
                champion = (v2, minimize_op, innat_equi_por, min_pol)

    return (champion)

### Game play  - Pure Strategy

In [5]:
####Op has been updated by maximizer, so minimizer react to the innate op after that
def minimizer_play(s,v1,max_opinion,n, max_touched): 
    
    op = copy.copy(s)
    #update innat opinion 
    op[v1,0] = max_opinion
    print('_______________________')
    print('Minimizer Play')
#     print('Only 1 opinion changed')
#     print(op)
    
    min_champion = choose_min_vertex(op, n, max_touched)
    (v2, min_opinion, innat_equi_por, min_pol) = min_champion
    
    if v2 == None:
        print('Minimizer fail')

    else:
        print("                                ")
        print("Minimizer finds its target agent:")
#         print('v2', 'changed_opinion', 'innate_obj', 'obj')
#         print(v2, min_opinion, innat_equi_por, min_pol)
        #Store innate_op of the min_selected vertex
        old_opinion_min = op[v2,0]
        ##### change the agent's opinion
        op[v2,0] = min_opinion   #-------------------------------------------------> store minimize strategy


        print("    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))

        print("Network reaches equilibrium Polarization: " + str(min_pol))
#         print('2 opinion changed')
#         print(op)

    return (v2,min_opinion, min_pol)

In [6]:
def maximizer_play(s,v2,min_opinion,n,min_touched): 
    op = copy.copy(s)
    #update innat opinion 
    op[v2,0] = min_opinion  # Op has been updated by minimizer, so maximizer react to the innate op after that
#     print('Only 1 opinion changed')
#     print(op)
    

    max_champion = choose_max_vertex(op, n, min_touched) # The best choice among all opinions and vertexs
    (v1, max_opinion, innate_obj, max_pol) = max_champion

    if v1 == None:
        print('Maximizer fail')

    else:
        print("                                ")
        print("Maximizer finds its target agent:")
#         print('v1', 'changed_opinion', 'innate_obj', 'obj')
#         print(max_champion)

        #Store innate_op of the max_selected vertex
        old_opinion_max = op[v1, 0]
        ##### change the agent's opinion
        op[v1,0] = max_opinion
        ## check if agent's opinionis is changed or not
        print("    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
#         print("Network reaches equilibrium Polarization: " + str(max_pol))
#         print('2 opinion changed')
#         print(op)

    return(v1, max_opinion, max_pol)